# K-Nearest Neighbors (KNN) from Scratch

## Introduction

The K-Nearest Neighbors (KNN) algorithm is a simple yet powerful supervised machine learning algorithm used for classification and regression tasks. It works by finding the k most similar instances (neighbors) to a given data point and making predictions based on the majority class among those neighbors.

In this notebook, we will implement the KNN algorithm from scratch using Python. We will go through the following steps:

1. **Import Libraries**: Import the necessary libraries for data manipulation and mathematical calculations.
2. **Load Dataset**: Load the Iris dataset and split it into training and test sets.
3. **Similarity**: Calculate the similarity between data instances using the Euclidean distance measure.
4. **Neighbors**: Identify the k most similar neighbors for a given test instance.
5. **Response**: Determine the majority class among the neighbors to make a prediction.
6. **Accuracy**: Evaluate the accuracy of the predictions by comparing them to the actual class labels.
7. **KNN Algorithm**: Combine all the steps to create the complete KNN algorithm and test it on the dataset.



In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



In [2]:
import csv

with open('../input/iris-dataset/iris.data.txt', 'r') as csvfile:

    lines = csv.reader(csvfile)

    for row in lines :

    	print (', '.join(row))

5.1, 3.5, 1.4, 0.2, Iris-setosa
4.9, 3.0, 1.4, 0.2, Iris-setosa
4.7, 3.2, 1.3, 0.2, Iris-setosa
4.6, 3.1, 1.5, 0.2, Iris-setosa
5.0, 3.6, 1.4, 0.2, Iris-setosa
5.4, 3.9, 1.7, 0.4, Iris-setosa
4.6, 3.4, 1.4, 0.3, Iris-setosa
5.0, 3.4, 1.5, 0.2, Iris-setosa
4.4, 2.9, 1.4, 0.2, Iris-setosa
4.9, 3.1, 1.5, 0.1, Iris-setosa
5.4, 3.7, 1.5, 0.2, Iris-setosa
4.8, 3.4, 1.6, 0.2, Iris-setosa
4.8, 3.0, 1.4, 0.1, Iris-setosa
4.3, 3.0, 1.1, 0.1, Iris-setosa
5.8, 4.0, 1.2, 0.2, Iris-setosa
5.7, 4.4, 1.5, 0.4, Iris-setosa
5.4, 3.9, 1.3, 0.4, Iris-setosa
5.1, 3.5, 1.4, 0.3, Iris-setosa
5.7, 3.8, 1.7, 0.3, Iris-setosa
5.1, 3.8, 1.5, 0.3, Iris-setosa
5.4, 3.4, 1.7, 0.2, Iris-setosa
5.1, 3.7, 1.5, 0.4, Iris-setosa
4.6, 3.6, 1.0, 0.2, Iris-setosa
5.1, 3.3, 1.7, 0.5, Iris-setosa
4.8, 3.4, 1.9, 0.2, Iris-setosa
5.0, 3.0, 1.6, 0.2, Iris-setosa
5.0, 3.4, 1.6, 0.4, Iris-setosa
5.2, 3.5, 1.5, 0.2, Iris-setosa
5.2, 3.4, 1.4, 0.2, Iris-setosa
4.7, 3.2, 1.6, 0.2, Iris-setosa
4.8, 3.1, 1.6, 0.2, Iris-setosa
5.4, 3.4

In [3]:
import random

In [4]:
def loadDataset(filename, split, trainingSet=[] , testSet=[]):

    with open(filename, 'r') as csvfile:

      lines = csv.reader(csvfile)

      dataset = list(lines)

      for x in range(len(dataset)-1):

        for y in range(4):

          dataset[x][y] = float(dataset[x][y])

        if random.random() < split:

          trainingSet.append(dataset[x])

        else:

          testSet.append(dataset[x])

In [5]:
trainingSet=[]

testSet=[]

loadDataset('../input/iris-dataset/iris.data.txt', 0.66, trainingSet, testSet)

print ('Train: ' + repr(len(trainingSet)))

print ('Test: ' + repr(len(testSet)) )

Train: 92
Test: 57


In [6]:
trainingSet[0]

[5.1, 3.5, 1.4, 0.2, 'Iris-setosa']

# 2. Similarity

To make predictions we need to calculate the similarity between any two given data instances. This is needed so that we can locate the k most similar data instances in the training dataset for a given member of the test dataset and in turn, make a prediction.

Given that all four flower measurements are numeric and have the same units, we can directly use the Euclidean distance measure. 

Additionally, we want to control which fields to include in the distance calculation. Specifically, we only want to include the first 4 attributes. One approach is to limit the Euclidean distance to a fixed length, ignoring the final dimension.

Putting all of this together, you have to define the Euclidean distance

In [7]:
import math


In [8]:

def euclideanDistance(instance1, instance2, length):
    
    return math.sqrt(sum([(instance1[i]-instance2[i])**2 for i in range(length)]))

In [9]:
data1 = [2, 2, 2, 'a']

data2 = [4, 4, 4, 'b']

distance = euclideanDistance(data1, data2, 3)

print ('Distance: ' , repr(distance))



Distance:  3.4641016151377544


In [10]:
def manhattanDistance(instance1, instance2, length):
    return sum([abs((instance1[i]-instance2[i])) for i in range(length)])

In [11]:
def minkowskiDistance(instance1, instance2, length, q):
    return (sum([abs((instance1[i]-instance2[i])) for i in range(length)]))**(1/float(q))

In [12]:
data1 = [2, 2, 2, 'a']

data2 = [4, 4, 4, 'b']

distance = manhattanDistance(data1, data2, 3)

print ('Distance: ' , repr(distance))


Distance:  6


In [13]:
data1 = [2, 2, 2, 'a']

data2 = [4, 4, 4, 'b']

distance = minkowskiDistance(data1, data2, 3, 2)

print ('Distance: ' , repr(distance))


Distance:  2.449489742783178


# 3. Neighbors

Now that we have a similarity measure, we can use it to collect the k most similar instances for a given unseen instance.

This is a straightforward process of calculating the distance for all instances and selecting a subset with the smallest distance values.

Below is the getNeighbors function that returns k most similar neighbors from the training set for a given test instance (using the already defined euclideanDistance function)

In [14]:
import operator

In [15]:


def getNeighbors(trainingSet, testInstance, k):

    distances = []

    length = len(testInstance)-1

    for x in range(len(trainingSet)):

        dist = euclideanDistance(testInstance, trainingSet[x], length)

        distances.append((trainingSet[x], dist))

    distances.sort(key=operator.itemgetter(1))

    neighbors = []

    for x in range(k):

        neighbors.append(distances[x][0])

    return neighbors

We can test out this function as follows:

In [16]:


trainSet = [[2, 2, 2, 'a'], [4, 4, 4, 'b']]

testInstance = [5, 5, 5]

k = 1

neighbors = getNeighbors(trainSet, testInstance, 1)

print(neighbors)

[[4, 4, 4, 'b']]


# 4. Response

Once we have located the most similar neighbors for a test instance, the next task is to devise a predicted response based on those neighbors.

We can do this by allowing each neighbor to vote for their class attribute, and taking the majority vote as the prediction.

Below is a function for getting the majority voted response from a number of neighbors. It assumes the class is the last attribute for each neighbor.

In [17]:

def getResponse(neighbors):

    classVotes = {}

    for x in range(len(neighbors)):

        response = neighbors[x][ -1 ] #complete with appropriate number

        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)

    return sortedVotes[0][0]

We can test out this function with some test neighbors, as follows:

In [18]:
neighbors = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]

response = getResponse(neighbors)

print(response)

a


# 5. Accuracy



We have all of the pieces of the KNN algorithm in place. An important remaining concern is how to evaluate the accuracy of predictions.

An easy way to evaluate the accuracy of the model is to calculate a ratio of the total correct predictions out of all predictions made, called classification accuracy.

Below is the getAccuracy function that sums the total correct predictions and returns the accuracy as a percentage of correct classifications.

In [19]:
def getAccuracy(testSet, predictions):
    
    correct = 0
    
    for i in range(len(predictions)):
        
        if predictions[i] == testSet[i][-1]:
            
            correct += 1
    
    return (correct/float(len(testSet))) * 100.0

We can test this function with a test dataset and predictions, as follows:



In [20]:
testSet = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]

predictions = ['a', 'a', 'a']

accuracy = getAccuracy(testSet, predictions)

print(accuracy)

66.66666666666666


In [21]:
def Knn(trainingSet, testSet, k):
    
    predictions = []
    for testInstance in testSet:
    
        neighbors = getNeighbors(trainingSet, testInstance, k)
        Response = getResponse(neighbors)
        predictions.append(Response)
    
    acc = getAccuracy(testSet, predictions)
    print("accuracy = ",acc)

In [22]:
Knn(trainingSet, testSet,1)

accuracy =  0.0
